In [36]:
import os
import sys
import pandas as pd
import numpy as np

# Image Generator
from keras.preprocessing.image import ImageDataGenerator

# Plotting
import matplotlib.pyplot as plt

# Modelling CNN
import tensorflow as tf
import keras
from keras import Model, Sequential
from keras.layers import Dropout, Conv2D, MaxPooling2D, ZeroPadding2D, Dense, Input, Flatten, BatchNormalization, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from numba import cuda

In [4]:
sys.path.append("../")
import data_paths as dp

D:\wildfire-sumatera-dataset already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-jpeg already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2 already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\landsat-8 already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-jpeg\sentinel-2 already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-jpeg\landsat-8 already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\landsat-8\prefire already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\landsat-8\postfire already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\prefire already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire already exist


# Constants

In [5]:
RANDOM_STATE = 42
BATCH_SIZE = 32
SEED = RANDOM_STATE
WIDTH = 256
HEIGHT = 256
CHANNEL = 3

# Prepare Data

In [7]:
# landsat-8
landsat_df = pd.read_csv(dp.METADATA_LANDSAT_8_FILE_PATH)
landsat_df_backup = landsat_df.copy()

In [8]:
# convert column 'class' type to string
landsat_df['class'] = landsat_df['class'].astype(str)
# filter image that is in a good condition
landsat_df = landsat_df[landsat_df['image_condition'] == 1]

In [9]:
# spliting data for testing and training
from sklearn.model_selection import train_test_split

# Spliting based on class
X_train, X_test, _, _, = train_test_split(
    landsat_df.index, 
    landsat_df['class'], 
    stratify = landsat_df['class'],
    test_size = 0.2,
    random_state = RANDOM_STATE
)

In [10]:
train_df = landsat_df.loc[X_train]
test_df = landsat_df.loc[X_test]

In [24]:
print('Class quantity in train df')
display(train_df['class'].value_counts())
print('Class quantity in test df')
display(test_df['class'].value_counts())

Class quantity in train df


1    3911
2    1282
0    1075
Name: class, dtype: int64

Class quantity in test df


1    978
2    321
0    269
Name: class, dtype: int64

# Create generator (Image Augmentation)

In [80]:
train_generator = ImageDataGenerator(
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range= 0.1,
    shear_range= 0.1,
    rescale= 1./255,
    validation_split=0.2,
)

test_generator = ImageDataGenerator(
    rescale= 1./255,
)

In [81]:
# Applying Generator by using data from dataframe
train_data = train_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = dp.LANDSAT_8_JPEG_FOLDER_PATH,
    x_col = 'file_paths_jpeg',
    y_col = 'class',
    batch_size = BATCH_SIZE,
    target_size = (WIDTH,HEIGHT),
    subset = "training",
    class_mode = "categorical",
)

validation_data = train_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = dp.LANDSAT_8_JPEG_FOLDER_PATH,
    x_col = 'file_paths_jpeg',
    y_col = 'class',
    batch_size = BATCH_SIZE,
    seed = SEED,
    target_size = (WIDTH,HEIGHT),
    subset = "validation",
    class_mode = "categorical",
)

test_data = test_generator.flow_from_dataframe(
    dataframe = test_df,
    directory = dp.LANDSAT_8_JPEG_FOLDER_PATH,
    x_col = 'file_paths_jpeg',
    y_col = 'class',
    batch_size = len(test_df.index),
    seed = SEED,
    target_size = (WIDTH,HEIGHT),
    class_mode = "categorical")

Found 5015 validated image filenames belonging to 3 classes.
Found 1253 validated image filenames belonging to 3 classes.
Found 1568 validated image filenames belonging to 3 classes.


### Create Model Prototype version 1

In [8]:
import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation


def get_model_ver_1(input_shape):
    # Create Sequential model
    model = Sequential()
    
    # Convolution 2D Layer with kernel size 3x3 (filters=32)
    model.add(Conv2D(filters=32, kernel_size=3, input_shape=input_shape))
    # Activation Layer ReLu function
    model.add(Activation('relu'))
    
    # Convolution 2D Layer with kernel size 3x3 (filters=64)
    model.add(Conv2D(filters=64, kernel_size=3))
    # Activation layer ReLu function
    model.add(Activation('relu'))
    # Max Pooling layer (2D) with pool size of 2x2 (default), strides=(2,2)
    model.add(MaxPooling2D(strides=(2,2)))
    
    # Convolution 2D Layer with kernel size 3x3 (filters=128)
    model.add(Conv2D(filters=128, kernel_size=3))
    # Activation layer ReLu function
    model.add(Activation('relu'))
    # Max Pooling layer (2D) with pool size of 2x2 (default), strides=(2,2)
    model.add(MaxPooling2D(strides=(2,2)))
    
    # Flatten layer
    model.add(Flatten())
    # Fully Connected Layer
    model.add(Dense(256))
    # Drop Out Layer
    model.add(Dropout(0.2))
    
    # Output model (multiclass)
    model.add(Dense(3))
    
    # Activation layer using Sigmoid Function for Logistic Regression
    model.add(Activation('softmax'))

    return model

In [9]:
# check if gpu is detected and ready to be used by tensorflow for neural network training process
if tf.test.gpu_device_name(): 
    print(f"Default GPU Device -> {tf.test.gpu_device_name()}")

else:
    print("Please install GPU version of TF")

Default GPU Device -> /device:GPU:0


In [ ]:
INPUT_SIZE = (256, 256, 3)
model = get_model_ver_1(INPUT_SIZE)

In [ ]:
# Reset Device and Free Memory from GPU
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
cnn_model = get_model_ver_1()

# Implement callbacks

In [25]:
callbacks_softmax = [
    EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 2),
    ModelCheckpoint(
        filepath = './traditional_cnn_multiclass_softmax.h5', 
        monitor = 'val_loss', 
        save_best_only = True,
        mode = 'min', 
        verbose = 2,
    )
]

cnn_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall'],
)

NameError: name 'traditional_cnn_softmax' is not defined

In [ ]:
with tf.device('/GPU:0'):
    history_softmax = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=10,
    steps_per_epoch=train_data.samples//BATCH_SIZE,
    validation_steps=valid_data.samples//BATCH_SIZE,
    workers=8,
    callbacks=callbacks_softmax) 

In [43]:
y_true = np.random.randint(3, size=5)


In [45]:
print(y_true)
y_pred = [
    [0.1, 0.5, 0.4],
    [0, 0.3, 0.7],
    [1, 0, 0],
    [0.2, 0.6, 0.2],
    [0.1, 0.6, 0.2],
]

[0 2 0 1 0]
[0 1 2 1 1 1 1 2 0 1]


In [33]:
tf.ones_like(y_true)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>

In [46]:
keras.metrics.categorical_accuracy(y_true, y_pred)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 0., 0., 1., 1.], dtype=float32)>